<a href="https://colab.research.google.com/github/cansibi/Deep-learning/blob/master/CNN-CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
import time

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [3]:
if torch.cuda.is_available():
  device='cuda:0'
else:
  device='cpu'
print(device)

cuda:0


In [4]:
BatchSize=16
Epoch=40
transform=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
traindata=torch.utils.data.DataLoader(trainset,batch_size=BatchSize,shuffle=True,num_workers=2)
Testset=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)
Testdata=torch.utils.data.DataLoader(Testset,batch_size=BatchSize,num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
CNNnet=ResNet18()
CNNnet.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inp

In [ ]:
import torch.optim as opt
LossFunction=nn.CrossEntropyLoss()
lr=1e-3
Optimizer=opt.Adam(CNNnet.parameters(),lr=lr)
count=0
for epoch in range(Epoch):
  loss=0.0
  time_start=time.time()
  if epoch==15:
    Optimizer=opt.Adam(CNNnet.parameters(),lr=1e-4)
  if epoch==30:
    Optimizer=opt.Adam(CNNnet.parameters(),lr=1e-5)
  for i,data in enumerate(traindata,0):
    Image,Label=data[0].to(device),data[1].to(device)
    Optimizer.zero_grad()
    Prediction=CNNnet(Image)
    EveryLoss=LossFunction(Prediction,Label)
    EveryLoss.backward()
    Optimizer.step()
    loss+=EveryLoss.item()
    count+=1
  print('epoch %d loss: %.3f' %
                  (epoch + 1, loss / count))   
  time_end=time.time()
  print('time cost',time_end-time_start,'s')
  count=0
print('over')

epoch 1 loss: 0.858
time cost 606.4539420604706 s


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in Testdata:
        Image,Label=data[0].to(device),data[1].to(device)
        outputs = CNNnet(Image)
        _, predicted = torch.max(outputs.data, 1)
        total += Label.size(0)
        correct += (predicted == Label).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in Testdata:
        Image,Label=data[0].to(device),data[1].to(device)
        outputs = CNNnet(Image)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == Label).squeeze()
        for i in range(4):
            label = Label[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))